In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

# Import required libraries
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow
import os
import yaml
import json

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-

In [ ]:
# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("14")

# download oblique dataset
dataset = version.download("yolov8")

print("Dataset location:", dataset.location)
print("Contents of dataset location:")
print(os.listdir(dataset.location))

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.69, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to test-14 in yolov8:: 100%|██████████| 23098/23098 [00:04<00:00, 5284.59it/s]


Dataset location: /content/test-14
Contents of dataset location:
['test', 'README.roboflow.txt', 'valid', 'data.yaml', 'README.dataset.txt', 'train']


In [ ]:
yaml_path = os.path.join(dataset.location, "data.yaml")

with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
print(data_config)

print("YAML Path:", yaml_path)

with open(original_yaml_path, 'r') as f:
    print(f.read())



{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/14', 'version': 14, 'workspace': 'capstone-ncwsi'}, 'test': '../test/images', 'train': 'test-14/train/images', 'val': 'test-14/valid/images'}
YAML Path: /content/test-14/data.yaml
names:
- Beetle-fire Tree
- Dead Tree
- debris
nc: 3
roboflow:
  license: CC BY 4.0
  project: test-5dvot
  url: https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/14
  version: 14
  workspace: capstone-ncwsi
test: ../test/images
train: test-14/train/images
val: test-14/valid/images



In [ ]:
import yaml

yaml_path = '/content/test-14/data.yaml'
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("YAML file updated.")

with open(yaml_path, 'r') as f:
    print(yaml.safe_load(f))

YAML file updated.
{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/14', 'version': 14, 'workspace': 'capstone-ncwsi'}, 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}


In [ ]:
import yaml

with open('/content/test-14/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
    print(data)

import os

print(os.listdir('/content/test-14'))
print(os.listdir('/content/test-14/train'))

{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/14', 'version': 14, 'workspace': 'capstone-ncwsi'}, 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}
['README.dataset.txt', 'valid', 'test', 'data.yaml', 'README.roboflow.txt', 'train']
['images', 'labels']


In [ ]:
import os

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files[:5]:
            print('{}{}'.format(subindent, f))
        if len(files) > 5:
            print('{}...'.format(subindent))

print_directory_structure('/content/test-14')

test-14/
    README.dataset.txt
    data.yaml
    README.roboflow.txt
    valid/
        images/
            DJI_0556_JPG_jpg.rf.ebe04b0c22881f1afe52728cb49c5d69.jpg
            DJI_0127_JPG_jpg.rf.413f5d165a2857756595b8c33e3d00b9.jpg
            DJI_0488_JPG_jpg.rf.ef424358175af7ed325c71f677cfefc9.jpg
            DJI_0899_JPG_jpg.rf.77111bd0d87421dfaed1363805c2c7fd.jpg
            DJI_0942_JPG_jpg.rf.fd60467a9b51c58a735f80f97e0e1348.jpg
            ...
        labels/
            DJI_0255_JPG_jpg.rf.de07ef20ea31feb593f6f47bffcdff20.txt
            DJI_0180_JPG_jpg.rf.4f733f4c5cef309296f5b4bf8ffaf87e.txt
            DJI_0191_JPG_jpg.rf.f2e56f72864c110c9a4b4db9b392ab79.txt
            DJI_0551_JPG_jpg.rf.0fc472aa4cb58a0effed348a4d50e55c.txt
            DJI_0568_JPG_jpg.rf.96ecb401673b18817859d0f37e12e4a1.txt
            ...
    test/
        images/
            DJI_0488_JPG_jpg.rf.bfb1c6184c6788949493c4f9e1b489c3.jpg
            DJI_0156_JPG_jpg.rf.0fc0584ed56bf5180ed6848a669676ad.jpg
 

In [ ]:
import random
from pathlib import Path
import yaml
import shutil
import os

def create_small_dataset(original_yaml_path, small_dataset_ratio=0.2):
    with open(original_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    base_dir = Path(original_yaml_path).parent
    original_train = base_dir / data['train']
    original_val = base_dir / data['val']

    small_train = original_train.parent / (original_train.name + '_small')
    small_val = original_val.parent / (original_val.name + '_small')

    for orig, small in [(original_train, small_train), (original_val, small_val)]:
        print(f"Processing {orig} to {small}")
        if orig.is_dir():
            os.makedirs(small, exist_ok=True)
            image_files = list(orig.glob('*.jpg')) + list(orig.glob('*.png'))
            print(f"Found {len(image_files)} images in {orig}")
            sampled_files = random.sample(image_files, max(1, int(len(image_files) * small_dataset_ratio)))
            print(f"Sampled {len(sampled_files)} images")

            for img_file in sampled_files:
                dest_img = small / img_file.name
                print(f"Copying {img_file} to {dest_img}")
                try:
                    shutil.copy(img_file, dest_img)
                except shutil.SameFileError:
                    print(f"Skipping {img_file} as it's the same as the destination")
                except Exception as e:
                    print(f"Error copying {img_file}: {e}")

                # copy labeled file
                label_file = orig.parent / 'labels' / (img_file.stem + '.txt')
                if label_file.exists():
                    os.makedirs(small.parent / 'labels', exist_ok=True)
                    dest_label = small.parent / 'labels' / (img_file.stem + '.txt')
                    print(f"Copying {label_file} to {dest_label}")
                    try:
                        shutil.copy(label_file, dest_label)
                    except shutil.SameFileError:
                        print(f"Skipping {label_file} as it's the same as the destination")
                    except Exception as e:
                        print(f"Error copying {label_file}: {e}")
                else:
                    print(f"Label file not found: {label_file}")
        else:
            print(f"Warning: {orig} is not a directory. Skipping.")

    small_yaml_path = base_dir / (Path(original_yaml_path).stem + '_small.yaml')
    data['train'] = str(small_train.relative_to(base_dir))
    data['val'] = str(small_val.relative_to(base_dir))

    with open(small_yaml_path, 'w') as f:
        yaml.dump(data, f)

    return str(small_yaml_path)


original_yaml_path = '/content/test-14/data.yaml'
small_yaml_path = create_small_dataset(original_yaml_path)
print(f"Small dataset created: {small_yaml_path}")

# varify small dataset
with open(small_yaml_path, 'r') as f:
    small_data = yaml.safe_load(f)
    print("Small dataset YAML content:", small_data)

base_dir = Path(original_yaml_path).parent
print("Small train directory content:", os.listdir(base_dir / small_data['train']))
print("Small valid directory content:", os.listdir(base_dir / small_data['val']))

流式输出内容被截断，只能显示最后 5000 行内容。
Skipping /content/test-14/train/labels/DJI_0488_JPG_jpg.rf.6b9d0c61b22da3857d0dd080117d07e9.txt as it's the same as the destination
Copying /content/test-14/train/images/DJI_0484_JPG_jpg.rf.1db98ffb1789e15972502055e55ba98a.jpg to /content/test-14/train/images_small/DJI_0484_JPG_jpg.rf.1db98ffb1789e15972502055e55ba98a.jpg
Copying /content/test-14/train/labels/DJI_0484_JPG_jpg.rf.1db98ffb1789e15972502055e55ba98a.txt to /content/test-14/train/labels/DJI_0484_JPG_jpg.rf.1db98ffb1789e15972502055e55ba98a.txt
Skipping /content/test-14/train/labels/DJI_0484_JPG_jpg.rf.1db98ffb1789e15972502055e55ba98a.txt as it's the same as the destination
Copying /content/test-14/train/images/DJI_0440_JPG_jpg.rf.eb0ca6852a6cdb1458c9f6cc488d065f.jpg to /content/test-14/train/images_small/DJI_0440_JPG_jpg.rf.eb0ca6852a6cdb1458c9f6cc488d065f.jpg
Copying /content/test-14/train/labels/DJI_0440_JPG_jpg.rf.eb0ca6852a6cdb1458c9f6cc488d065f.txt to /content/test-14/train/labels/DJI_0440_JPG_j

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from pathlib import Path
import yaml
import logging
import os

# set log
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomYOLO(YOLO):
    def __init__(self, model='yolov8m.pt', task=None):
        super().__init__(model, task)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def _load(self, weights='yolov8m.pt', task='detect'):
        super()._load(weights, task)
        # modify focal loss
        if hasattr(self.model, 'model') and hasattr(self.model.model, 'loss'):
            original_loss_func = self.model.model.loss
            def custom_loss(*args, **kwargs):
                loss, loss_items = original_loss_func(*args, **kwargs)
                # replace to focal loss
                pred_scores, target_scores = args[0][1], args[1]['cls']
                focal_loss = self.focal_loss(pred_scores, target_scores)
                loss_items[1] = focal_loss  
                loss = loss_items.sum()
                return loss, loss_items
            self.model.model.loss = custom_loss

def grid_search_focal_loss(yaml_path, save_dir, alphas, gammas, epochs=10):
    results = {}
    for alpha in alphas:
        for gamma in gammas:
            logging.info(f"Testing alpha={alpha}, gamma={gamma}")
            try:
                model = CustomYOLO('yolov8m.pt')
                model.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)

                train_results = model.train(
                    data=yaml_path,
                    epochs=epochs,
                    imgsz=640,
                    batch=16,
                    lr0=0.001,
                    lrf=0.01,
                    momentum=0.937,
                    weight_decay=0.001,
                    warmup_epochs=5,
                    warmup_bias_lr=0.1,
                    warmup_momentum=0.8,
                    box=10.0,
                    cls=1.0,
                    dfl=1.5,
                    patience=100,
                    optimizer="AdamW",
                    cos_lr=True,
                    mixup=0.5,
                    copy_paste=0.3,
                    degrees=15.0,
                    hsv_h=0.015,
                    hsv_s=0.7,
                    hsv_v=0.4,
                    translate=0.2,
                    scale=0.6,
                    shear=3.0,
                    perspective=0.0005,
                    flipud=0.5,
                    fliplr=0.5,
                    mosaic=1.0,
                    conf=0.001,
                    iou=0.5,
                    verbose=True,
                    save_period=10,
                    label_smoothing=0.1,
                    project=str(save_dir),
                    name=f'focal_loss_a{alpha}_g{gamma}',
                    exist_ok=True
                )

                # validate results
                val_results = model.val(data=yaml_path)

                results[(alpha, gamma)] = {
                    'mAP50': val_results.box.map50,
                    'mAP50-95': val_results.box.map,
                    'precision': val_results.box.p,
                    'recall': val_results.box.r
                }

                logging.info(f"Finished alpha={alpha}, gamma={gamma}")

                
                with open(save_dir / 'intermediate_results.yaml', 'w') as f:
                    yaml.dump(results, f)

            except Exception as e:
                logging.error(f"Error occurred for alpha={alpha}, gamma={gamma}: {str(e)}")

    return results

if __name__ == "__main__":
    yaml_path = '/content/test-14/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search')

    
    os.makedirs(save_dir, exist_ok=True)

    alphas = [0.25, 0.5, 0.75]
    gammas = [1.0, 2.0, 3.0]

    search_results = grid_search_focal_loss(yaml_path, save_dir, alphas, gammas)

    # save final results
    with open(save_dir / 'final_results.yaml', 'w') as f:
        yaml.dump(search_results, f)

    logging.info("Grid search completed. Results saved.")


Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/test-14/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search, name=focal_loss_a0.25_g1.0, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fal

100%|██████████| 755k/755k [00:00<00:00, 74.5MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 288MB/s]


AMP: checks passed ✅


...


10 epochs completed in 0.591 hours.
Optimizer stripped from /content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0/weights/best.pt...
Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:10<00:00,  2.83it/s]


                   all        961       3376      0.408      0.399      0.347      0.109
      Beetle-fire Tree        170        232      0.445      0.496      0.434      0.137
             Dead Tree        559       1347      0.387      0.408      0.344       0.11
                debris        411       1797      0.393      0.293      0.264     0.0812
Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0
Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:14<00:00,  4.08it/s]


                   all        961       3376      0.406        0.4      0.347      0.108
      Beetle-fire Tree        170        232      0.444      0.496      0.435      0.135
             Dead Tree        559       1347      0.387      0.411      0.344       0.11
                debris        411       1797      0.388      0.293      0.262     0.0806
Speed: 0.2ms preprocess, 9.3ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search/focal_loss_a0.75_g3.0


In [ ]:
!pip install ruamel.yaml
import matplotlib.pyplot as plt
import numpy as np
import yaml
from ruamel.yaml import YAML
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

# load results
def load_results(file_path):
    yaml = YAML(typ='unsafe')
    with open(file_path, 'r') as f:
        return yaml.load(f)


# analyze results
def analyze_results(results):
    print("Starting analysis...")
    metrics = ['mAP50', 'mAP50-95', 'precision', 'recall']
    alphas = sorted(set(a for a, _ in results.keys()))
    gammas = sorted(set(g for _, g in results.keys()))

    print("Metrics:", metrics)
    print("Alphas:", alphas)
    print("Gammas:", gammas)

    for metric in metrics:
        plt.figure(figsize=(12, 8))

        for alpha in alphas:
            x = [g for a, g in results.keys() if a == alpha]
            y = [results[(a, g)][metric] for a, g in results.keys() if a == alpha]
            if isinstance(y[0], (np.ndarray, list)):
                y = [np.mean(v) if isinstance(v, np.ndarray) else sum(v)/len(v) for v in y]
            plt.plot(x, y, marker='o', label=f'alpha={alpha}')

        plt.title(f'{metric} vs Gamma for different Alpha values')
        plt.xlabel('Gamma')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(True)
        plt.savefig(f'{metric}_vs_gamma.png')
        plt.close()

    
    for metric in metrics:
        plt.figure(figsize=(10, 8))
        heatmap_data = np.zeros((len(alphas), len(gammas)))

        for i, alpha in enumerate(alphas):
            for j, gamma in enumerate(gammas):
                value = results[(alpha, gamma)][metric]
                if isinstance(value, (np.ndarray, list)):
                    value = np.mean(value) if isinstance(value, np.ndarray) else sum(value)/len(value)
                heatmap_data[i, j] = value

        plt.imshow(heatmap_data, cmap='hot', interpolation='nearest')
        plt.colorbar()
        plt.title(f'{metric} Heatmap')
        plt.xlabel('Gamma')
        plt.ylabel('Alpha')
        plt.xticks(range(len(gammas)), gammas)
        plt.yticks(range(len(alphas)), alphas)

        for i in range(len(alphas)):
            for j in range(len(gammas)):
                plt.text(j, i, f'{heatmap_data[i, j]:.2f}',
                         ha="center", va="center", color="w")

        plt.savefig(f'{metric}_heatmap.png')
        plt.close()

    # find the best parameters
    best_params = {}
    for metric in metrics:
        best_combo = max(results.items(), key=lambda x: np.mean(x[1][metric]) if isinstance(x[1][metric], (np.ndarray, list)) else x[1][metric])
        best_value = np.mean(best_combo[1][metric]) if isinstance(best_combo[1][metric], (np.ndarray, list)) else best_combo[1][metric]
        best_params[metric] = {'alpha': best_combo[0][0], 'gamma': best_combo[0][1], 'value': best_value}

    print("Best parameter combinations for each metric:")
    for metric, params in best_params.items():
        print(f"{metric}: Alpha = {params['alpha']}, Gamma = {params['gamma']}, Value = {params['value']:.4f}")


save_dir = Path('/content/drive/MyDrive/YOLOv8_focal_loss_checkpoints/grid_search')
results_file = save_dir / 'final_results.yaml'

print("Checking if results file exists:", results_file.exists())

if results_file.exists():
    print("Results file found. Loading and analyzing results:")
    search_results = load_results(results_file)
    analyze_results(search_results)
else:
    print(f"Results file not found: {results_file}")
    print("Please check the file path and ensure the grid search results were saved.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 29.8 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking if results file exists: True
Results file found. Loading and analyzing results:
Starting analysis...
Metrics: ['mAP50', 'mAP50-95', 'precision', 'recall']
Alphas: [0.25, 0.5, 0.75]
Gammas: [1.0, 2.0, 3.0]
Best parameter combinations for each metric:
mAP50: Alpha = 0.25, Gamma = 1.0, Value = 0.3473
mAP50-95: Alpha = 0.25, Gamma = 1.0, Value = 0.1083
precision: Alpha = 0.25, Gamma = 1.0, Value = 0.4064
recall: Alpha = 0.25, Gamma = 1.0, Value = 0.3998


In [19]:
from google.colab import drive
drive.flush_and_unmount()

!rm -rf /content/drive



Drive not mounted, so nothing to flush and unmount.


In [20]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122


import os
import yaml
import json
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow

# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("14")

# download oblique dataset
dataset = version.download("yolov8")

# Now we can use dataset.location
yaml_path = os.path.join(dataset.location, "data.yaml")

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files[:5]:  # Limiting to first 5 files to avoid clutter
            print('{}{}'.format(subindent, f))
        if len(files) > 5:
            print('{}...'.format(subindent))

print("Directory structure:")
print_directory_structure(dataset.location)

def fix_yaml_paths(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)


    base_dir = os.path.dirname(yaml_path)
    data['train'] = os.path.relpath(os.path.join(base_dir, 'train', 'images'), base_dir)
    data['val'] = os.path.relpath(os.path.join(base_dir, 'valid', 'images'), base_dir)
    data['test'] = os.path.relpath(os.path.join(base_dir, 'test', 'images'), base_dir)

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    print("YAML file updated.")


fix_yaml_paths(yaml_path)

print("\nYAML file content:")
with open(yaml_path, 'r') as f:
    print(f.read())

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomYOLO(YOLO):
    def __init__(self, model='yolov8m.pt', task=None):
        super().__init__(model, task)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def _load(self, weights='yolov8n.pt', task='detect'):
        super()._load(weights, task)

        if hasattr(self.model, 'model') and hasattr(self.model.model, 'loss'):
            original_loss_func = self.model.model.loss
            def custom_loss(*args, **kwargs):
                loss, loss_items = original_loss_func(*args, **kwargs)

                pred_scores, target_scores = args[0][1], args[1]['cls']
                focal_loss = self.focal_loss(pred_scores, target_scores)
                loss_items[1] = focal_loss
                loss = loss_items.sum()
                return loss, loss_items
            self.model.model.loss = custom_loss

best_alpha = 0.25
best_gamma = 1.0

def train_and_evaluate_with_focal_loss(yaml_path, save_dir, alpha, gamma):
    model = CustomYOLO('yolov8m.pt')
    model.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)

    results = model.train(
        data=yaml_path,
        epochs=200,
        imgsz=640,
        batch=16,
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=5,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        box=10.0,
        cls=1.0,
        dfl=1.5,
        patience=100,
        optimizer="AdamW",
        cos_lr=True,
        mixup=0.5,
        copy_paste=0.3,
        degrees=15.0,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.2,
        scale=0.6,
        shear=3.0,
        perspective=0.0005,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        conf=0.001,
        iou=0.5,
        verbose=True,
        save_period=5,
        label_smoothing=0.1,
        project=str(save_dir),
        name='focal_loss_final',
        exist_ok=True
    )

    # evaluate model
    val_results = model.val(data=yaml_path)

    return results, val_results


save_dir = Path('/content/drive/MyDrive/YOLOv8_focal_loss')

results, val_results = train_and_evaluate_with_focal_loss(yaml_path, save_dir, best_alpha, best_gamma)

print(f"Final validation results:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.p:.4f}")
print(f"Recall: {val_results.box.r:.4f}")

Mounted at /content/drive
Looking in indexes: https://download.pytorch.org/whl/cu122
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.69, to fix: `pip install ultralytics==8.0.196`
Directory structure:
test-14/
    README.roboflow.txt
    data.yaml
    README.dataset.txt
    test/
        images/
            DJI_0489_JPG_jpg.rf.f61742b8afa1fbad907c9b187af83a72.jpg
            DJI_0535_JPG_jpg.rf.52c3acbbbb0205ae608d32baa84c643f.jpg
            DJI_0488_JPG_jpg.rf.bfb1c6184c6788949493c4f9e1b489c3.jpg
            DJI_0434_JPG_jpg.rf.604b26638de53bd1a3994615164554bb.jpg
            DJI_0925_JPG_jpg.rf.f54ade141717c27f50412fcae4209dc8.jpg
            ...
        labels/
            DJI_0548_JPG_jpg.rf.aaba6b8ed28c5573aa94278a4aa9dfef.txt
            DJI_0326_JPG_jpg.rf.5e10e88ed328fec7b0e8a4da9ca51638.txt
            DJI_0581_JPG_jpg.rf.0bd6f862826a7242e76e9c76a6a16335.txt
            DJI_0539_JPG_jpg.rf.bdbc15ef579

train: Scanning /content/test-14/train/labels.cache... 10092 images, 2259 backgrounds, 0 corrupt: 100%|██████████| 10092/10092 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/YOLOv8_focal_loss/focal_loss_final/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.001), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/YOLOv8_focal_loss/focal_loss_final
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:06<00:00,  4.82it/s]


                   all        961       3376      0.492      0.433      0.416      0.148
      Beetle-fire Tree        170        232       0.57      0.517       0.51      0.193
             Dead Tree        559       1347      0.455      0.459      0.417      0.144
                debris        411       1797       0.45      0.322      0.321      0.106
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_focal_loss/focal_loss_final
Ultralytics YOLOv8.2.69 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:07<00:00,  8.05it/s]


                   all        961       3376      0.491      0.434      0.415      0.148
      Beetle-fire Tree        170        232      0.569      0.517      0.509      0.193
             Dead Tree        559       1347      0.456      0.461      0.416      0.144
                debris        411       1797      0.449      0.324       0.32      0.106
Speed: 0.1ms preprocess, 3.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_focal_loss/focal_loss_final
Final validation results:
mAP50: 0.4150
mAP50-95: 0.1479


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [2]:
def calculate_metrics(results):
    print("\nFinal validation results:")
    print(f"mAP50: {results.box.map50:.4f}")
    print(f"mAP50-95: {results.box.map:.4f}")

    # Calculate overall metrics
    if hasattr(results.box, 'r') and isinstance(results.box.r, np.ndarray):
        overall_mar = np.mean(results.box.r)
        print(f"Overall mAR: {overall_mar:.4f}")
    else:
        print("Overall mAR: Not available")

    if hasattr(results.box, 'p') and isinstance(results.box.p, np.ndarray):
        overall_precision = np.mean(results.box.p)
        print(f"Overall Precision: {overall_precision:.4f}")
    else:
        print("Overall Precision: Not available")

    if hasattr(results.box, 'r') and isinstance(results.box.r, np.ndarray):
        overall_recall = np.mean(results.box.r)
        print(f"Overall Recall: {overall_recall:.4f}")
    else:
        print("Overall Recall: Not available")
    

    # Calculate class wise metrics
    print("\nClass-wise results:")
    if hasattr(results, 'names') and isinstance(results.box.r, np.ndarray):
        class_recalls = results.box.r
        class_precisions = results.box.p if isinstance(results.box.p, np.ndarray) else []
        for i, c in enumerate(results.names):
            print(f"Class {i} ({c}):")
            if i < len(class_recalls):
                class_recall = class_recalls[i]
                print(f"  Recall: {class_recall:.4f}")
                print(f"  mAR: {class_recall:.4f}")  # mAR is equal to Recall
            if i < len(class_precisions):
                class_precision = class_precisions[i]
                print(f"  Precision: {class_precision:.4f}")
    else:
        print("Class-wise results not available in the expected format.")

    print("\nAdditional Information:")
    for key, value in vars(results).items():
        if key not in ['box', 'names', 'results_dict']:
            print(f"{key}: {value}")

    if hasattr(results, 'confusion_matrix'):
        print("\nConfusion Matrix:")
        print(results.confusion_matrix)

# Calculate and print metrics
calculate_metrics(results)


Final validation results:
mAP50: 0.3937
mAP50-95: 0.1476
Overall mAR: 0.4025
Overall Precision: 0.5054
Overall Recall: 0.4025

Class-wise results:
Class 0 (0):
  Recall: 0.5086
  mAR: 0.5086
  Precision: 0.5904
Class 1 (1):
  Recall: 0.4224
  mAR: 0.4224
  Precision: 0.4600
Class 2 (2):
  Recall: 0.2766
  mAR: 0.2766
  Precision: 0.4658

Additional Information:
save_dir: runs/detect/val
plot: True
on_plot: <bound method BaseValidator.on_plot of <ultralytics.models.yolo.detect.val.DetectionValidator object at 0x79b6fc8570d0>>
speed: {'preprocess': 0.3248494075810872, 'inference': 2.7575445720978258, 'loss': 0.0007631379285290388, 'postprocess': 3.9030438779420087}
task: detect
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79b7ec9b7670>

Confusion Matrix:
